# Create and populate the 4Growth database

In this notebook, we will create and populate the 4Growth database. The database will be created using the `sglalchemy` library. The data model has been defined in this [document](https://docs.google.com/document/d/1gy7cM_QqEHODvdV_c6yTxIO-BMF3decttnxtpPZSPpg/edit?usp=sharing). The data will be populated using the survey samples provided in the this [folder](https://drive.google.com/drive/folders/1DXfhdd0O2vK_C9cbv0NNdrdJsV5tm8xd?usp=drive_link).

## Table of contents
### 1. [Create the database](#section_1)
### 2. [Define the data model using SQLAlchemy ORM](#section_2)
### 3. [Connect to the PostgreSQL Database](#section_3)
### 4. [Create the tables using SQLAlchemy](#section_4)
### 5. [Insert data into the tables](#section_5)
### 6. [Query the database](#section_6)



## Setup

### Library import


In [1]:
import os
from datetime import datetime

import pandas as pd
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import TIMESTAMP, Column, ForeignKey, Integer, String, create_engine
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get the environment variables
database_user = os.getenv("DATABASE_USER")
database_password = os.getenv("DATABASE_PASSWORD")
database_host = os.getenv("DATABASE_HOST")
database_port = os.getenv("DATABASE_PORT")

### Utils

In [3]:
def create_database(dbname: str, user: str, password: str, host: str, port: str):
    """
    Create the database if it doesn't exist
    """
    try:
        # Connect to the 'postgres' database to check for survey_db
        conn = psycopg2.connect(
            dbname="postgres", user=user, password=password, host=host, port=port
        )
        conn.autocommit = True
        cur = conn.cursor()

        # Check if the database already exists
        cur.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{dbname}';")
        exists = cur.fetchone()

        if not exists:
            # Create the survey_db if it doesn't exist
            cur.execute(f"CREATE DATABASE {dbname};")
            print(f"Database '{dbname}' created successfully!")
        else:
            print(f"Database '{dbname}' already exists.")

        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error creating database: {e}")

In [4]:
def read_survey_data(file_path: str) -> pd.DataFrame:
    """
    Read the survey data from the Excel file
    """
    df = pd.read_excel(
        file_path,
        sheet_name="The Grid (general)",
        header=2,
    )
    columns = df.columns
    df = df[columns[:3]]
    df.rename(columns={columns[0]: "ID", columns[1]: "Question"}, inplace=True)
    df = df[df["ID"].str.contains(r"\.", regex=True, na=False)]
    return df

In [5]:
def convert_first_row_to_columns(df):
    """
    Convert the first row of the DataFrame to columns
    """
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df.reset_index(drop=True, inplace=True)
    return df

<a id='section_1'></a>
## 1. Create the database

In [6]:
create_database(
    dbname="db_4growth",
    user=database_user,
    password=database_password,
    host=database_host,
    port=database_port,
)

Database 'db_4growth' created successfully!


<a id='section_2'></a>
## 2. Define the data model using SQLAlchemy ORM

SQLAlchemy uses classes to represent database tables, which map directly to the columns in the 
database.

In [7]:
# Base class for all models
Base = declarative_base()


# Define SurveyID model
class SurveyID(Base):
    """
    SurveyID model
    """

    __tablename__ = "SurveyID"
    ID = Column(Integer, primary_key=True, autoincrement=True)
    Name = Column(String(255))
    Description = Column(String(255))
    LastUpdated = Column(TIMESTAMP, default=datetime.now)

    answers = relationship("Answer", back_populates="survey")


# Define Question model
class Question(Base):
    """
    Question model
    """

    __tablename__ = "Question"
    ID = Column(Integer, primary_key=True, autoincrement=True)
    Name = Column(String(255))
    Description = Column(String(255))
    LastUpdated = Column(TIMESTAMP, default=datetime.now)

    answers = relationship("Answer", back_populates="question")


# Define Categorical_answers model
class CategoricalAnswers(Base):
    """
    Categorical_answers model
    """

    __tablename__ = "Categorical_answers"
    ID = Column(Integer, primary_key=True, autoincrement=True)
    Name = Column(String(255))
    Description = Column(String(2000))
    LastUpdated = Column(TIMESTAMP, default=datetime.now)

    answers = relationship("Answer", back_populates="categorical_answer")


# Define Answer model
class Answer(Base):
    """
    Answer model
    """

    __tablename__ = "Answer"
    ID = Column(Integer, primary_key=True, autoincrement=True)
    SurveyID = Column(Integer, ForeignKey("SurveyID.ID"), nullable=False)
    Question = Column(Integer, ForeignKey("Question.ID"), nullable=False)
    Categorical_answer = Column(Integer, ForeignKey("Categorical_answers.ID"))
    Open_ended_answer = Column(String(255))

    survey = relationship("SurveyID", back_populates="answers")
    question = relationship("Question", back_populates="answers")
    categorical_answer = relationship("CategoricalAnswers", back_populates="answers")

<a id='section_3'></a>
## 3. Connect to the PostgreSQL Database

In [8]:
# Create an engine and connect to the PostgreSQL database
DATABASE_URL = f"postgresql://{database_user}:{database_password}@{database_host}:{database_port}/\
db_4growth"
engine = create_engine(DATABASE_URL)

<a id='section_4'></a>
## 4. Create the tables using SQLAlchemy

In [9]:
# Create a session factory
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
# Create all tables in the database
Base.metadata.create_all(engine)

<a id='section_5'></a>
## 5. Insert data into the tables

### Insert data into the `SurveyID` table

**Get survey names from the mock data folder**

In [11]:
folder_path = "../data/raw/mock_data/"
entries = os.listdir(folder_path)
survey_names = [entry.split(".")[0].split("_")[-1] for entry in entries]
survey_names

['YannyLaurel',
 'Ruuta Skujina',
 'AntonioVage',
 'MichaelJagermeister',
 'Daire Boyle',
 'AitorTiya',
 'EmiliaDuco']

**Insert survey names into the `SurveyID` table**

In [12]:
for survey_name in survey_names:
    survey = SurveyID(Name=survey_name)
    session.add(survey)
    session.commit()

### Insert data into the `Question` table

**Get questions from the survey template**

In [13]:
df = read_survey_data("../data/raw/27062024_4Growth_AssessmentGrid_FINAL_mock_data.xlsx")
df

,ID,Question,Answer
0,1.1,Organisation Name:,NaN
1,1.2,Sector (Agri/Forestry/Both):,(Sector)
2,1.3,Type of stakeholder:,(Type of stakeholder)
3,1.4,Location (Country/Region):,(EU Member State)
4,1.5,Primary Area of Operation in agriculture:,(in agriculture)
...,...,...,...
91,10.5,Do you use digital technologies to track and e...,(Yes/No/Don't know)
93,11.1,Are there plans to expand or upgrade your curr...,(Yes/No/Don't know)
94,11.2,What would help facilitate the expansion/upgra...,(Future expansion/upgrade)
95,11.3,What type of developments do you anticipate in...,(Answer 3)


**Insert questions into the `Question` table**

In [ ]:
for name, question in df[["ID", "Question"]].itertuples(index=False, name=None):
    question = Question(Name=name, Description=question)  # noqa: PLW2901
    session.add(question)
    session.commit()

### Insert data into the `Categorical_answers` table

**Get categorical questions from the survey template**

In [15]:
df_categorical = df.dropna(subset=["Answer"]).drop(columns=["Question"])
df_categorical.drop_duplicates(subset=["Answer"], inplace=True)

**Get categorical answers**

In [16]:
df_data = pd.read_excel(
    "../data/raw/27062024_4Growth_AssessmentGrid_FINAL_mock_data.xlsx",
    sheet_name="Data",
)

df_data_1 = df_data.iloc[:27]

df_data_2 = df_data.iloc[28:39]
df_data_2 = convert_first_row_to_columns(df_data_2)
df_data_2

df_data_3 = df_data.iloc[41:]
df_data_3 = convert_first_row_to_columns(df_data_3)

df_answers = pd.concat([df_data_1, df_data_2, df_data_3], axis=1)

df_answers.head()

,(Sector),(Type of stakeholder),(EU Member State),(in agriculture),(in forestry),(Yes/No),(Type of device),(Future expansion/upgrade),(Yes/No/Don't know),(Data sharing practices),...,(Savings in inputs),(Job creation impact),(Overall Social impact),(Effects on biodiversity),(Answer 3),(Types of tools or platforms),(Answer 4),Type of data (receive or provide),(Data storage),(Governance model)
0,Agriculture,Farmer/agricultural producers,Austria,Crop cultivation- grains,Reforestation,Yes,Desktop computers,Better connectivity/Infrastructure,Yes,Open sharing,...,Significant,Substantial impact,Positive,Positive,Advancements in existing technologies,Field Data Collection Apps,<25%,Farm-level data,On-premises servers/local storage facilities,Traditional/Subsistence
1,Forestry,Forester,Belgium,Crop cultivation- vegetables,"Forest conservation - thinning, pruning, weed ...",No,Laptop computers,More income/Access to funding,No,Restricted sharing,...,Moderate,Moderate impact,Neutral,Negative,Emergence of new technologies,Precision Agriculture Technology,<50%,Earth Observation (EO) data,Cloud-based platforms,Cooperative
2,Both,Forest owner,Bulgaria,Crop cultivation- legumes,Felling,NaN,Tablets,Standardisation efforts/Regulatory support,Don't know,No sharing,...,Minimal,Negligible impact,Negative,No impact,No significant changes anticipated,IoT Devices,<75%,Environmental data,Data warehouses,Corporate
3,NaN,Forest operator,Croatia,Crop cultivation- fruits,Transportation of logs,NaN,Smartphones,Better training and education,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Remote Sensing Platforms,>90%,Socio-economic data,Agricultural information management systems,Contract Farming or Forestry
4,NaN,Forest product processor,Cyprus,Plant propagation,Non-Timber Forest Products (NTFPs),NaN,GPS devices,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Farm Management Software,NaN,Supply chain data,Geographic Information Systems (GIS),Community Supported


**Insert categorical answers into the `Categorical_answers` table**

In [17]:
for group in df_categorical["Answer"].to_list():
    answers = df_answers[group].dropna().to_list()
    for answer in answers:
        categorical_answers = CategoricalAnswers(Name=answer, Description=group)
        session.add(categorical_answers)
        session.commit()

### Insert data into the `Answer` table

**Get answers from the mock data folder**

In [18]:
folder_path = "../data/raw/mock_data/"
entries = os.listdir(folder_path)
survey_names = [entry.split(".")[0].split("_")[-1] for entry in entries]
survey_names

['YannyLaurel',
 'Ruuta Skujina',
 'AntonioVage',
 'MichaelJagermeister',
 'Daire Boyle',
 'AitorTiya',
 'EmiliaDuco']

In [19]:
df_template = read_survey_data("../data/raw/27062024_4Growth_AssessmentGrid_FINAL_mock_data.xlsx")

In [20]:
# Create a session factory
Session = sessionmaker(bind=engine)
session = Session()

for entry in entries:
    survey_name = entry.split(".")[0].split("_")[-1]
    df = read_survey_data(os.path.join(folder_path, entry))
    survey = session.query(SurveyID).filter_by(Name=survey_name).first()

    for question_name, answer_group in df_template[["ID", "Answer"]].itertuples(
        index=False, name=None
    ):
        question = session.query(Question).filter_by(Name=question_name).first()
        if not df[df["ID"] == question_name].empty:
            answer = df[df["ID"] == question_name]["Answer"].iloc[0]

            if str(answer_group) == "nan":
                answer_row = Answer(
                    SurveyID=survey.ID, Question=question.ID, Open_ended_answer=answer
                )
                session.add(answer_row)
                session.commit()

            else:
                categorical_answer = (
                    session.query(CategoricalAnswers)
                    .filter_by(Name=str(answer), Description=answer_group)
                    .first()
                )

                if categorical_answer is not None:
                    answer_row = Answer(
                        SurveyID=survey.ID,
                        Question=question.ID,
                        Categorical_answer=categorical_answer.ID,
                    )
                    session.add(answer_row)
                    session.commit()

/home/iker/mambaforge/envs/4_growth/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/home/iker/mambaforge/envs/4_growth/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/home/iker/mambaforge/envs/4_growth/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/home/iker/mambaforge/envs/4_growth/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/home/iker/mambaforge/envs/4_growth/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supp

<a id='section_6'></a>
## 6. Query the database

**Query Data from `SurveyID` table**

In [19]:
surveys = session.query(SurveyID).all()

print("\nSurveyID Table Data:")
for survey in surveys:
    print(f"ID: {survey.ID}, Name: {survey.Name}")


SurveyID Table Data:
ID: 1, Name: YannyLaurel
ID: 2, Name: Ruuta Skujina
ID: 3, Name: AntonioVage
ID: 4, Name: MichaelJagermeister
ID: 5, Name: Daire Boyle
ID: 6, Name: Lefteris Mamais
ID: 7, Name: AitorTiya
ID: 8, Name: EmiliaDuco
ID: 9, Name: Dimitri Papadakis
